In [ ]:
# Подготовка изображений
from PIL import Image
import os.path
import glob

# функция конвертации и приведение размеров к 300х300
def convertjpg(jpgfile,outdir,width=300,height=300):
    img=Image.open(jpgfile)
    try:
        new_img=img.resize((width,height),Image.BILINEAR)   
        new_img.save(os.path.join(outdir,os.path.basename(jpgfile)))
    except Exception as e:
        print(e)
# применение функции ко всем изображениям в папке Train
for jpgfile in glob.glob('C:/Users/Артем/Desktop/Python Nr/Nr_test1/Train'):
    convertjpg(jpgfile,'C:/Users/Артем/Desktop/Python Nr/Nr_test1/Train')

In [ ]:
from tensorflow import keras
from keras.applications.inception_v3 import InceptionV3
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
import os
from keras import optimizers
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3,preprocess_input
from keras.layers import GlobalAveragePooling2D,Dense
from keras.models import Model
from keras.utils.vis_utils import plot_model
from keras.optimizers import Adagrad
 
 
 # подключение директорий
train_dir = 'C:/Users/Артем/Desktop/Python Nr/Nr_test1/Train'
val_dir = 'C:/Users/Артем/Desktop/Python Nr/Nr_test1/Val'
 
 # Генерация массивов по изображению
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
 
 # Генерация массивов по изображению с улучшеными параметрами
# train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=30., width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
# val_datagen = ImageDataGenerator(rescale=1./255, rotation_range=30., width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
 
 # Создание тензора для набора изображений
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(300, 300), batch_size=10, class_mode='binary')
val_generator = train_datagen.flow_from_directory(val_dir, target_size=(300, 300), batch_size=10, class_mode='binary')
 
# resnet_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
 # Построение базовой модели InceptionV3
base_model = InceptionV3(weights='imagenet',include_top=False, input_shape=(300, 300, 3))
 
 # Добавление выходного слоя
x = base_model.output
 # GlobalAveragePooling2D преобразует тензор MxNxC в тензор 1xC, где C - количество каналов
x = GlobalAveragePooling2D()(x) 
x = Dense(512,activation='relu')(x)
predictions = Dense(1,activation='sigmoid')(x)
model = Model(inputs=base_model.input,outputs=predictions)
 
'''
 Функции для старта обучения
'''
def setup_to_transfer_learning(model,base_model):
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
 
def setup_to_fine_tune(model,base_model):
    GAP_LAYER = 17 
    for layer in base_model.layers[:GAP_LAYER+1]:
        layer.trainable = False
    for layer in base_model.layers[GAP_LAYER+1:]:
        layer.trainable = True
    model.compile(optimizer=Adagrad(learning_rate=0.0001),loss='binary_crossentropy',metrics=['accuracy'])
    
 # Обучение модели при помощи встроенной функции model.fit
setup_to_transfer_learning(model,base_model)
history_tl = model.fit(x=train_generator,
                       validation_data=val_generator,
                       validation_steps=5,
                       epochs=5,
                    )
model.save('C:/Users/Артем/Desktop/Python Nr/Nr_test1/fire-inceptionV3-more21.h5')
setup_to_fine_tune(model,base_model)
history_ft = model.fit(x=train_generator,
                       validation_data=val_generator,
                       validation_steps=5,
                       epochs=5,
                                )
model.save('C:/Users/Артем/Desktop/Python Nr/Nr_test1/fire-inceptionV3-more22.h5')

In [4]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
from keras import optimizers
import matplotlib.pyplot as plt
from keras.applications import VGG16
 
from tensorflow import keras
from keras import models
import numpy as np
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
import os
import tkinter as tk
from tkinter import filedialog

from PIL import Image
import os.path
import glob
import keras.utils as image


 # Загрузка обученной модели
model = models.load_model('C:/Users/Артем/Desktop/Python Nr/Nr_test1/fire-inceptionV3-more21.h5')
 
root = tk.Tk()
root.withdraw()
 
 # Поочередная загрузка, конвертация и проверка размеров для всех фото из папки
 # Применение модели к каждому фото и вывод результатов на экран
for file in glob.glob('C:/Users/Артем/Desktop/Python Nr/Nr_test1/Test_images/*.png'):
    img = Image.open(file)
    img = img.resize((300, 300))
    #img = img.scale()
    x = image.img_to_array(img)
    x /= 255
    x = np.expand_dims(x, axis=0)
    #x = preprocess_input(x, mode='torch')
    y = model.predict(x)
    y = (1 - y) * 100
    y = y[0][0]
    print(file, "=" ,y, "%")

1/1 [==============================] - 1s 865ms/step
C:/Users/Артем/Desktop/Python Nr/Nr_test1/Test_images\fire.1.png = 99.81403 %
1/1 [==============================] - 0s 76ms/step
C:/Users/Артем/Desktop/Python Nr/Nr_test1/Test_images\fire.2.png = 96.84736 %
1/1 [==============================] - 0s 82ms/step
C:/Users/Артем/Desktop/Python Nr/Nr_test1/Test_images\fire.543.png = 99.63992 %
1/1 [==============================] - 0s 80ms/step
C:/Users/Артем/Desktop/Python Nr/Nr_test1/Test_images\Fire_Ex.png = 60.755074 %
1/1 [==============================] - 0s 84ms/step
C:/Users/Артем/Desktop/Python Nr/Nr_test1/Test_images\Fire_Ex2.png = 80.40349 %
1/1 [==============================] - 0s 75ms/step
C:/Users/Артем/Desktop/Python Nr/Nr_test1/Test_images\non_fire.24.png = 0.0015854836 %
1/1 [==============================] - 0s 75ms/step
C:/Users/Артем/Desktop/Python Nr/Nr_test1/Test_images\non_fire.25.png = 0.002247095 %
1/1 [==============================] - 0s 76ms/step
C:/Users/Артем

TensorFlow version: 2.11.0
